In [1]:
import numpy as np
from numpy import linalg as LA
import pandas as pd

import json
import keras
import tensorflow
import tensorflow_addons as tfa
from keras import layers, Model
from keras.models import Sequential
from keras.applications import DenseNet201
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
csv_path = './UrbanSound8K/metadata/UrbanSound8K.csv'
spec_path = "./numpySpectrograms/"
model_save_path = "ovr_basemodel_extractor"
extractor_weights_path = "basemodel-known"
test_size = 0.2
val_size = 0.2
batch_size = 32
num_classes = 5

In [3]:
class SpecLoader(keras.utils.Sequence):
  def __init__(self, x_set, y_set, batch_size, spec_dir):
    self.x, self.y = x_set, y_set
    self.batch_size = batch_size
    self.spec_dir = spec_dir

  def __len__(self):
    return int(np.ceil(len(self.x) / self.batch_size))

  def __getitem__(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) *
    self.batch_size]
    batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

    batchSpecs = []
    for fileName in batch_x:
        spec = np.load(self.spec_dir + fileName + ".npy")
        batchSpecs.append(spec.transpose())
    return np.array(batchSpecs), np.array(batch_y)

In [4]:
data_df = pd.read_csv(csv_path)
data_df_known = data_df.loc[data_df["classID"] < num_classes]
data_df_unknown = data_df.loc[data_df["classID"] >= num_classes]

X_trainval, X_test, y_trainval, y_test = train_test_split(data_df_known['slice_file_name'].tolist(), data_df_known['classID'].tolist(), test_size=test_size, random_state = 42)

X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=val_size, random_state = 42)
X_trash, X_unknown, y_trash, y_unknown = train_test_split(data_df_unknown['slice_file_name'].tolist(), data_df_unknown['classID'].tolist(), test_size=test_size, random_state = 42)
y_unknown = [num_classes] * len(y_unknown)

X_test = X_test + X_unknown
y_test = y_test + y_unknown
test_loader = SpecLoader(X_test, y_test, batch_size, spec_path)

# 2D array of loaders. Train and val loader for each known class
loaders=[]
for i in range(num_classes):
    loaders.append([])
    ovr_y_train = [1 if j == i else 0 for j in y_train]
    ovr_y_val = [1 if j == i else 0 for j in y_val]
    loaders[i].append(SpecLoader(X_train, ovr_y_train, batch_size, spec_path))
    loaders[i].append(SpecLoader(X_val, ovr_y_val, batch_size, spec_path))

In [5]:
densenet = DenseNet201(
            include_top=False,
            weights="imagenet",
            input_tensor=None,
            input_shape=None,
            pooling="avg")
prev_model = Sequential()
prev_model.add(densenet)
prev_model.add(layers.Dense(num_classes, activation="softmax"))
prev_model.load_weights(extractor_weights_path)
feature_extractor = prev_model.layers[0]
feature_extractor.trainable = False

# Array of models. One for each known class
models=[]
for i in range(num_classes):
    model = Sequential()
    model.add(feature_extractor)
    model.add(layers.Dense(2, activation="softmax"))
    model.summary()
    model.compile(
          optimizer="Adam",
          loss="sparse_categorical_crossentropy",
          metrics=["accuracy"])
    models.append(model)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Model)          (None, 1920)              18321984  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 3842      
Total params: 18,325,826
Trainable params: 3,842
Non-trainable params: 18,321,984
_________________________________________________________________
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Model)          (None, 1920)              18321984  
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 3842      
Total params: 18,325,826
Trainable params: 3,842
Non-trainable params: 18,321,984
_________________________________________________________________


In [6]:
for i in range(num_classes):
    earlystopping = EarlyStopping(
                        patience=5, 
                        restore_best_weights=True)
    checkpoint = ModelCheckpoint(
                        model_save_path+str(i), 
                        monitor="val_accuracy", 
                        save_best_only=True)

    models[i].fit(x=loaders[i][0],
              validation_data=loaders[i][1],
              callbacks=[checkpoint, earlystopping],
              epochs=70,
              verbose=1
             )

Epoch 1/70
89/89 [==============================] - 32s 360ms/step - loss: 0.1207 - accuracy: 0.9502 - val_loss: 0.0064 - val_accuracy: 0.9873
Epoch 2/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0209 - accuracy: 0.9961 - val_loss: 0.0031 - val_accuracy: 0.9873
Epoch 3/70
89/89 [==============================] - 9s 99ms/step - loss: 0.0154 - accuracy: 0.9972 - val_loss: 9.3250e-04 - val_accuracy: 0.9873
Epoch 4/70
89/89 [==============================] - 9s 99ms/step - loss: 0.0127 - accuracy: 0.9975 - val_loss: 4.4162e-04 - val_accuracy: 0.9873
Epoch 5/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0107 - accuracy: 0.9972 - val_loss: 2.7580e-04 - val_accuracy: 0.9873
Epoch 6/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0094 - accuracy: 0.9975 - val_loss: 1.4377e-04 - val_accuracy: 0.9859
Epoch 7/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0087 - accuracy: 0.9975 - val_loss: 1.2792e-04 - val_accuracy

Epoch 10/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0152 - accuracy: 0.9944 - val_loss: 1.0736e-04 - val_accuracy: 0.9676
Epoch 11/70
89/89 [==============================] - 9s 100ms/step - loss: 0.0144 - accuracy: 0.9947 - val_loss: 8.8869e-05 - val_accuracy: 0.9704
Epoch 12/70
89/89 [==============================] - 9s 99ms/step - loss: 0.0136 - accuracy: 0.9951 - val_loss: 1.0319e-04 - val_accuracy: 0.9718
Epoch 13/70
89/89 [==============================] - 9s 99ms/step - loss: 0.0137 - accuracy: 0.9944 - val_loss: 7.2088e-05 - val_accuracy: 0.9718
Epoch 14/70
89/89 [==============================] - 9s 99ms/step - loss: 0.0143 - accuracy: 0.9944 - val_loss: 1.4203e-04 - val_accuracy: 0.9760
Epoch 15/70
89/89 [==============================] - 9s 100ms/step - loss: 0.0145 - accuracy: 0.9947 - val_loss: 5.6737e-05 - val_accuracy: 0.9732
Epoch 16/70
89/89 [==============================] - 9s 100ms/step - loss: 0.0124 - accuracy: 0.9958 - val_loss: 3.9121e-0

89/89 [==============================] - 9s 98ms/step - loss: 0.0063 - accuracy: 0.9979 - val_loss: 1.0705e-05 - val_accuracy: 0.9845
Epoch 46/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0061 - accuracy: 0.9986 - val_loss: 1.2612e-05 - val_accuracy: 0.9831
Epoch 47/70
89/89 [==============================] - 9s 99ms/step - loss: 0.0064 - accuracy: 0.9979 - val_loss: 9.1552e-06 - val_accuracy: 0.9831
Epoch 48/70
89/89 [==============================] - 9s 99ms/step - loss: 0.0065 - accuracy: 0.9975 - val_loss: 9.0598e-06 - val_accuracy: 0.9873
Epoch 49/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0070 - accuracy: 0.9975 - val_loss: 7.0095e-06 - val_accuracy: 0.9845
Epoch 50/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0059 - accuracy: 0.9975 - val_loss: 8.0346e-06 - val_accuracy: 0.9859
Epoch 51/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0073 - accuracy: 0.9968 - val_loss: 6.3180e-06 - val_accurac

Epoch 32/70
89/89 [==============================] - 9s 99ms/step - loss: 0.0015 - accuracy: 0.9996 - val_loss: 7.9236e-04 - val_accuracy: 0.9873
Epoch 33/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0015 - accuracy: 0.9993 - val_loss: 7.7157e-04 - val_accuracy: 0.9859
Epoch 34/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0016 - accuracy: 0.9993 - val_loss: 5.8141e-04 - val_accuracy: 0.9845
Epoch 35/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0014 - accuracy: 0.9996 - val_loss: 6.5732e-04 - val_accuracy: 0.9873
Epoch 36/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0013 - accuracy: 0.9993 - val_loss: 5.2587e-04 - val_accuracy: 0.9859
Epoch 37/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 5.8404e-04 - val_accuracy: 0.9873
Epoch 38/70
89/89 [==============================] - 9s 98ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 5.2514e-04 -

In [7]:
def evaluate(predicted, expected):
    acc = np.mean(np.array(predicted) == np.array(expected))
    print("Overall accuracy: {}".format(acc))
    acc_dict = {}
    for i in range(len(expected)):
        expected_class = expected[i]
        if expected_class not in acc_dict:
            acc_dict[expected_class] = [0, 0]
        acc_dict[expected_class][1] += 1
        if expected_class == predicted[i]:
            acc_dict[expected_class][0] += 1
    for k,v in acc_dict.items():
        print("Accuracy for class {}: {}".format(k, v[0]/v[1]))
    for average in ["macro", "weighted", "micro"]:
        f1 = f1_score(expected, predicted, average=average)
        print("{} f1 score: {}".format(average, f1))

In [8]:
for i in range(num_classes):
    models[i].load_weights(model_save_path+str(i))

In [9]:
y_predicted = []
y_random = []
for data, labels in tqdm(test_loader):
    for i in range(len(data)):
        probabilities = []
        for j in range(num_classes):
            probabilities.append(models[j].predict(x=np.array([data[i]]))[0][1])
        best_class = np.argmax(probabilities)
        highest_prob = probabilities[best_class]
        if highest_prob < 0.5:
            y_predicted.append(num_classes)
        else:
            y_predicted.append(best_class)
        y_random.append(np.random.randint(6))

100%|██████████| 55/55 [06:51<00:00,  7.48s/it]


In [10]:
evaluate(y_predicted, y_test)
evaluate(y_random, y_test)

Overall accuracy: 0.5924441900400687
Accuracy for class 2: 0.9605911330049262
Accuracy for class 4: 0.9767441860465116
Accuracy for class 0: 0.9665071770334929
Accuracy for class 3: 0.8903508771929824
Accuracy for class 1: 0.9864864864864865
Accuracy for class 5: 0.22531939605110338
macro f1 score: 0.647683959407468
weighted f1 score: 0.5372796523877724
micro f1 score: 0.5924441900400687
Overall accuracy: 0.17515741270749857
Accuracy for class 2: 0.21182266009852216
Accuracy for class 4: 0.18023255813953487
Accuracy for class 0: 0.11961722488038277
Accuracy for class 3: 0.17543859649122806
Accuracy for class 1: 0.17567567567567569
Accuracy for class 5: 0.17886178861788618
macro f1 score: 0.149865961765545
weighted f1 score: 0.19862234667833067
micro f1 score: 0.17515741270749857
